<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1LeagueScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Module

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
from contextlib import closing
from operator import attrgetter
from pprint import pprint
from subprocess import DEVNULL
from subprocess import Popen
import time


# Team Dic

In [2]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmpDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126','名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmp2Dic = {'清水':'126', '福岡':'135'}

J12021TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '徳島':'toku', '福岡':'fuku', '鳥栖':'tosu', '大分':'oita'}

def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

get_keys_from_value(TeamDic,'276')

['札幌']

<select id="comparison1_team"><option value="0">チーム選択</option><option value="276">札幌</option><option value="31219">八戸</option><option value="30673">岩手</option><option value="238">仙台</option><option value="30310">秋田</option><option value="294">山形</option><option value="30674">福島</option><option value="120">鹿島</option><option value="176">水戸</option><option value="30305">栃木</option><option value="30103">群馬</option><option value="122">浦和</option><option value="199">大宮</option><option value="121">千葉</option><option value="132">柏</option><option value="270">FC東京</option><option value="123">東京Ｖ</option><option value="30532">町田</option><option value="86">川崎Ｆ</option><option value="124">横浜FM</option><option value="296">横浜FC</option><option value="30675">YS横浜</option><option value="130">湘南</option><option value="30676">相模原</option><option value="150">甲府</option><option value="30528">松本</option><option value="30535">長野</option><option value="193">新潟</option><option value="30308">富山</option><option value="30526">金沢</option><option value="126">清水</option><option value="131">磐田</option><option value="30677">藤枝</option><option value="31296">沼津</option><option value="127">名古屋</option><option value="30302">岐阜</option><option value="134">京都</option><option value="128">Ｇ大阪</option><option value="133">Ｃ大阪</option><option value="136">神戸</option><option value="30000">鳥取</option><option value="30314">岡山</option><option value="129">広島</option><option value="30851">山口</option><option value="30536">讃岐</option><option value="30116">徳島</option><option value="30148">愛媛</option><option value="30974">今治</option><option value="135">福岡</option><option value="30313">北九州</option><option value="269">鳥栖</option><option value="30531">長崎</option><option value="30303">熊本</option><option value="207">大分</option><option value="31508">宮崎</option><option value="31042">鹿児島</option><option value="30317">琉球</option></select>

# ScrapeData

In [ ]:
def scrapeData(homeTeamID,awayTeamID,year):
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チーム毎の平均スタッツを取得
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0,2,4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)

    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'home','Unnamed: 3': 'Data','試合平均.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)

    #試合結果を取得
    html = requests.get(url)

    html.encoding = "UTF-8"
    soup = BeautifulSoup(html.text, "html.parser")

    texts = soup.find('div', attrs={'class': 'gameTbl'}).text
    texts = re.findall(r'\w+', texts)
    time.sleep(1) 

    return df,texts

### def get last 3 match data

In [ ]:
def matchData1GamesBefore(TeamDic, year):
    data = pd.DataFrame()
    for homeTeamID in tqdm(TeamDic.values()):
        for awayTeamID in tqdm(TeamDic.values()):
            if ((homeTeamID == '126' and awayTeamID == '135') or (homeTeamID == '135' and awayTeamID == '126')):
                pass
            elif (homeTeamID != awayTeamID ):
                df,texts = scrapeData(homeTeamID,awayTeamID,year)
                #被りをなくす様にフラグを生成
                homeFlg = True
                #1試合前の情報
                if (texts[0] == year):
                    matchDate = texts[0] + '-' + texts[1] + '-' + texts[2] + '-' + texts[3] + ':' + texts[4]

                    if (homeTeamID !=TeamDic[texts[7]]):
                        homeGoal = texts[6]
                        homeTeam = texts[8]
                        awayGoal = texts[5]
                        awayTeam = texts[7]
                        homeFlg = False
                    else:
                        homeGoal = texts[5]
                        homeTeam = texts[7]
                        awayGoal = texts[6]
                        awayTeam = texts[8]

                    df.insert(0, 'スコア', [homeGoal,awayGoal])
                    df.insert(0, 'チーム名',[homeTeam,awayTeam])
                    df.insert(0, '試合日', matchDate)

                    time.sleep(1) 
                    if (homeFlg):
                        data = pd.concat([data, df]) 
            else:
                continue
    return data

In [ ]:
def matchData2GamesBefore(TeamDic, year):
    data = pd.DataFrame()
    for homeTeamID in tqdm(TeamDic.values()):
        for awayTeamID in TeamDic.values():
            if ((homeTeamID == '126' and awayTeamID == '135') or (homeTeamID == '135' and awayTeamID == '126')):
                pass
            elif (homeTeamID != awayTeamID):
                df,texts = scrapeData(homeTeamID,awayTeamID,year)
                #被りをなくす様にフラグを生成
                homeFlg = True
                #2試合前の情報
                if (texts[10] == year):
                    matchDate = texts[10] + '-' + texts[11] + '-' + texts[12] + '-' + texts[13] + ':' + texts[14]

                    if (homeTeamID !=TeamDic[texts[17]]):
                        homeGoal = texts[16]
                        homeTeam = texts[18]
                        awayGoal = texts[15]
                        awayTeam = texts[17]
                        homeFlg = False
                    else:
                        homeGoal = texts[15]
                        homeTeam = texts[17]
                        awayGoal = texts[16]
                        awayTeam = texts[18]
                    
                    df.insert(0, 'スコア', [homeGoal,awayGoal])
                    df.insert(0, 'チーム名',[homeTeam,awayTeam])
                    df.insert(0, '試合日', matchDate)

                    time.sleep(1) 
                    if (homeFlg):
                        data = pd.concat([data, df]) 
            else:
                continue
    return data

In [ ]:
def matchData3GamesBefore(TeamDic, year):
    data = pd.DataFrame()
    for homeTeamID in tqdm(TeamDic.values()):
        for awayTeamID in TeamDic.values():
            if ((homeTeamID == '126' and awayTeamID == '135') or (homeTeamID == '135' and awayTeamID == '126')):
                pass
            elif (homeTeamID != awayTeamID):
                df,texts = scrapeData(homeTeamID,awayTeamID,year)
                #被りをなくす様にフラグを生成
                homeFlg = True
                #3試合前の情報
                if (texts[20] == year):
                    matchDate = texts[20] + '-' + texts[21] + '-' + texts[22] + '-' + texts[23] + ':' + texts[24]

                    if (homeTeamID !=TeamDic[texts[27]]):
                        homeGoal = texts[26]
                        homeTeam = texts[28]
                        awayGoal = texts[25]
                        awayTeam = texts[27]
                        homeFlg = False
                    else:
                        homeGoal = texts[25]
                        homeTeam = texts[27]
                        awayGoal = texts[26]
                        awayTeam = texts[28]

                    df.insert(0, 'スコア', [homeGoal,awayGoal])
                    df.insert(0, 'チーム名',[homeTeam,awayTeam])
                    df.insert(0, '試合日', matchDate)

                    if (homeFlg):
                        data = pd.concat([data, df]) 
            else:
                continue
    return data

### scrape

In [ ]:
df1 = matchData1GamesBefore(tmpDic,'2021')
df1

In [ ]:
df1 = matchData1GamesBefore(TeamDic,'2021')
df2 = matchData2GamesBefore(TeamDic,'2021')

In [ ]:
df = pd.concat([df1,df2])
df

In [ ]:
from google.colab import files
import datetime

d_today = datetime.date.today()
dt = d_today.strftime('%Y%m%d')
filename =  'scrapeData' + dt + '.csv'
df.to_csv(filename, encoding = 'utf-8-sig') 
files.download(filename)

In [ ]:
df3 = matchData2GamesBefore(TeamDic,'2020')

In [ ]:
df4 = matchData3GamesBefore(TeamDic,'2020')

In [ ]:
tmpdf = pd.concat([df3,df4])
tmpdf

In [ ]:
filename =  'scrapeData' + dt + '_2' + '.csv'
tmpdf.to_csv(filename, encoding = 'utf-8-sig') 
files.download(filename)

'20210809'